In [1]:
import pandas as pd
import numpy as np
sf3 = pd.read_csv('data/SHARADAR_holdings.csv')
sf3 = sf3.dropna()

In [2]:
df=sf3

df[['ticker', 'investorname', 'calendardate']] = df[['ticker', 'investorname', 'calendardate']].astype(str)
df[['value', 'units', 'price']] = df[['value', 'units', 'price']].astype(float)
df=df[df['securitytype']=='SHR']
df=df[['ticker', 'investorname', 'calendardate', 'value', 'units', 'price']]

In [3]:
df_pivot = df.pivot_table(index=['ticker', 'investorname', 'calendardate'])

In [4]:
df_pivot

price    units   value
ticker     investorname             calendardate                        
0001388994 HUNTINGTON NATIONAL BANK 2018-09-30      0.0      1.0     0.0
                                    2018-12-31      0.0      1.0     0.0
                                    2019-06-30      0.0      1.0     0.0
                                    2019-09-30      0.0      1.0     0.0
                                    2019-12-31      0.0      1.0     0.0
...                                                 ...      ...     ...
ZZZOF      IFP ADVISORS INC         2021-09-30      7.0    130.0  1000.0
                                    2021-12-31      7.0    130.0  1000.0
           INVERNESS COUNSEL LLC    2018-03-31      0.0  23800.0  7000.0
                                    2018-06-30      0.0  23800.0  4000.0
                                    2018-09-30      0.0  23800.0  2000.0

[33563204 rows x 3 columns]

In [5]:
# Get unique counts of 'ticker', 'investorname', 'calendardate'
num_tickers = df['ticker'].nunique()
num_investors = df['investorname'].nunique()
num_dates = df['calendardate'].nunique()

# Reshape the values of the DataFrame to a 3D numpy array
#array_3d = df_pivot.values.reshape(num_tickers, num_investors, num_dates, 3)
num_tickers*num_investors*num_dates
df_pivot.shape
#df_pivot.values.reshape(num_tickers, num_investors, num_dates)
#num_tickers*num_investors*num_dates

(33563204, 3)

In [6]:

# Get the number of unique tickers, investors, and dates
num_tickers = df['ticker'].nunique()
num_investors = df['investorname'].nunique()
num_dates = df['calendardate'].nunique()

# Create a 3D numpy array of the same shape
array_3d = np.full((num_tickers, num_investors, num_dates), 0.0,dtype=np.float64)

# Create mappings from tickers, investors, and dates to indices
ticker_to_index = {ticker: i for i, ticker in enumerate(df['ticker'].unique())}
investor_to_index = {investor: i for i, investor in enumerate(df['investorname'].unique())}
date_to_index = {date: i for i, date in enumerate(df['calendardate'].unique())}

# Iterate over the DataFrame rows to fill in the values
for idx, row in df.iterrows():
    array_3d[ticker_to_index[row['ticker']], investor_to_index[row['investorname']], date_to_index[row['calendardate']]] = row['value']

KeyboardInterrupt: 

In [10]:
array_3d.shape
sum(sum(array_3d>0))

array([1281501, 1241164, 1267310, 1307288, 1320178, 1233265, 1229288,
       1204525, 1137732, 1043144, 1022793,  986188, 1056254, 1009221,
       1020553, 1031131, 1009939, 1014160, 1007388,  995732,  990469,
        920711,  924838,  924592,  934334,  898762,  902088,  776529,
             0,       0,       0,       0])